In [1]:
pip install pyspark

     --------------------                   170.5/317.0 MB 7.0 MB/s eta 0:00:22
Note: you may need to restart the kernel to use updated packages.


ERROR: Exception:
Traceback (most recent call last):
  File "c:\Users\Om Bade\AppData\Local\Programs\Python\Python310\lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "c:\Users\Om Bade\AppData\Local\Programs\Python\Python310\lib\site-packages\pip\_vendor\urllib3\response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
  File "c:\Users\Om Bade\AppData\Local\Programs\Python\Python310\lib\site-packages\pip\_vendor\urllib3\response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
  File "c:\Users\Om Bade\AppData\Local\Programs\Python\Python310\lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 98, in read
    data: bytes = self.__fp.read(amt)
  File "c:\Users\Om Bade\AppData\Local\Programs\Python\Python310\lib\http\client.py", line 465, in read
    s = self.fp.read(amt)
  File "c:\Users\Om Bade\AppData\Local\Programs\Python\Python310\lib\socket.py", l

In [2]:
import pyspark
from pyspark.sql import SparkSession

ModuleNotFoundError: No module named 'pyspark'

In [ ]:
spark = SparkSession.builder.appName('crew_requirement').getOrCreate()

In [ ]:
df = spark.read.csv('/content/cruise_ship_info.csv', inferSchema = True, header = True)

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/content/cruise_ship_info.csv.

In [ ]:
df.show(5)

In [ ]:
df.printSchema()

In [ ]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol = 'Cruise_line', outputCol = "Cruise_line_index")

df_indexed = indexer.fit(df).transform(df)

In [ ]:
df_indexed.show(10)

In [ ]:
df_indexed.select('Cruise_line', 'Cruise_line_index').distinct().show()

In [ ]:
df_indexed.groupby("Cruise_line").count().show()

In [ ]:
df_indexed.columns

In [ ]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols = ['Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'Cruise_line_index'], outputCol = 'features')

output = assembler.transform(df_indexed)

In [ ]:
output.show()

In [ ]:
train_data , test_data = output.randomSplit([.8,.2])
train_data.describe().show()

In [ ]:
from pyspark.ml.regression import LinearRegression
crew_req = LinearRegression(featuresCol= 'features', labelCol = 'crew')

model = crew_req.fit(train_data)



In [ ]:
result = model.evaluate(train_data)

In [ ]:
result.r2

In [ ]:
pred = model.transform(test_data)

In [ ]:
result_test = model.evaluate(test_data)

In [ ]:
result_test.r2

In [ ]:
pred.show()

In [ ]:
pip install ucimlrepo

In [ ]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
adult = fetch_ucirepo(id=2)

# data (as pandas dataframes)
X = adult.data.features
y = adult.data.targets

# metadata
print(adult.metadata)

# variable information
print(adult.variables)


In [ ]:
X.head()

In [ ]:
data = spark.read.csv('/content/adult.data')
data.show()

In [ ]:
labels = ['age', 'workclass', 'fnlwgt', 'education', 'numbers', 'marital', 'occupation', 'relation', 'race', 'gender', 'gain', 'loss', 'hourlypay', 'country', 'income']

In [ ]:
df = data.toDF(*labels)


In [ ]:
df.show(5)

In [ ]:
df.printSchema()

In [ ]:
from pyspark.sql.functions import col
new_df = df.withColumn('age' , col('age').cast('integer'))

In [ ]:
new_df.printSchema()

In [ ]:

for i in [ 'fnlwgt' , 'numbers', 'gain', 'loss', 'hourlypay']:
  new_df = new_df.withColumn(i , col(i).cast('integer'))

In [ ]:
from pyspark.sql.functions import *

new_df.select([count(when(col(c).isNull(), c )).alias(c) for c in new_df.columns]).show()

In [ ]:
df.select('workclass').distinct().show()

In [ ]:
df = new_df.replace(" ?" , None)

In [ ]:
from pyspark.sql.functions import *

df.select([count(when(col(c).isNull(), c )).alias(c) for c in df.columns]).show()

In [ ]:
df.groupby('occupation').count().show()

In [ ]:
df = df.fillna(" United-States", subset = ['country'])

In [ ]:
df = df.fillna(" Private", subset = ['workclass'])

In [ ]:
df = df.fillna(" Prof-specialty", subset = ['occupation'])

In [ ]:
df.select([count(when(col(c).isNull(), c )).alias(c) for c in df.columns]).show()

In [ ]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer , VectorAssembler



In [ ]:
df.columns

In [ ]:
categorical_cols = [ 'workclass',
 'education',
 'marital',
 'occupation',
 'relation',
 'race',
 'gender',
 'country']
numeric_cols = [ 'age' ,'fnlwgt' , 'numbers', 'gain', 'loss', 'hourlypay']
label = 'income'


In [ ]:
indexer = [StringIndexer(inputCol = c , outputCol = f"{c}_index", handleInvalid = 'keep') for c in categorical_cols]


In [ ]:
label_indexer = StringIndexer(inputCol = 'income', outputCol = 'label', handleInvalid = 'keep')

In [ ]:
assembler = VectorAssembler(inputCols = [f'{c}_index' for c in categorical_cols] + numeric_cols, outputCol = 'features')

In [ ]:
lr = LogisticRegression(featuresCol = 'features', labelCol = 'label')


In [ ]:
pipeline = Pipeline(stages = indexer + [assembler , label_indexer , lr] )

In [ ]:
train_data , test_data = df.randomSplit([.8, .2])

In [ ]:
model = pipeline.fit(train_data)

In [ ]:
prediction = model.transform(test_data)

In [ ]:
prediction.select('label', 'prediction').show()

In [ ]:
prediction.groupby('label', 'prediction').count().show()

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(predictionCol = 'prediction', labelCol = 'label', metricName = 'accuracy')

In [ ]:
evaluator.evaluate(prediction)